In [ ]:
####################################################################
# --- Setup: instala dependencias do Google (GenAI e Agentes)  --- #
####################################################################
!pip install -q google-genai
!pip install -q google-adk

In [17]:
#########################################################################################################
# --- Setup: import de libraries e set de variaveis de ambiente necessarias para execucao do agente --- #
#########################################################################################################5
import os
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
from google.colab import userdata

# Set variavel de ambiente com a secret de api key do gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

warnings.filterwarnings("ignore")

In [27]:
################################################################
# --- Utils: Executa o agente e retona resultado em string --- #
################################################################
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [28]:
##############################################
# --- Utils: Função que retorna markdown --- #
##############################################
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
##############################################
# --- Agente 1: Especialista em Nutricao --- #
##############################################
def agente_planejador_nutricao(calorias_diarias: str, preferencias: str, refeicoes: str) -> str:
    agente = Agent(
        name="agente_nutricionista",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em nutrição e planejamento de refeições.
        Com base nas seguintes informações fornecidas pelo usuário,
        crie um cardápio detalhado para 7 dias,
        levando em consideração a praticidade para preparar as refeições
        diariamente ou em formato de marmitas para a semana inteira.
        """,
        description="Agente de criação de cardápio baseado numa quantidade total de calorias, preferências alimentares e total de refeições",
        tools=[google_search]
    )

    prompt = f"""
    Informações do Usuário:
    - Meta de calorias diárias: [{calorias_diarias}]
    - Preferências alimentares: [{preferencias}]
    - Refeições a serem planejadas: [{refeicoes}]

    O cardápio deve:
    - Apresentar sugestões específicas para cada refeição em cada dia da semana.
    - As refeições podem se repetir de forma intercalada (exemplo: segunda, quarta e sexta as mesmas refeições), afim de facilitar o planejamento e execução das receitas.
    - Garantir que o total de calorias diárias fique próximo de [{calorias_diarias}].
    - Ser prático e fácil de seguir, com opções que podem ser preparadas rapidamente ou em grande quantidade para a semana.
    - Os igredientes da dieta devem ser baratos para comprar (não devem ser igredientes refinados).
    - Variar as opções ao longo da semana para garantir uma dieta equilibrada.
    - Você deve incluir o peso e calorias de cada alimento no planejamento do cardápio.

    Formato de saída desejado:

    **Cardápio Semanal:**

    **📆 Dia 1:**
    - **[REFEIÇÃO 1]: **
    - Total de calorias: [TOTAL CALORIAS];
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias);
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias)
    - **[REFEIÇÃO 2]: **
    - Total de calorias: [TOTAL CALORIAS]
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias);
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias)
    - ...

    **📆 Dia 2:**
    - **[REFEIÇÃO 1]:**
    - Total de calorias: [TOTAL CALORIAS];
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias);
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias)
    - **[REFEIÇÃO 2]:**
    - Total de calorias: [TOTAL CALORIAS];
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias);
      - [ALIMENTO] ([PESO] peso) ([CALORIAS] calorias)
    - ...

    ... e assim por diante para os 7 dias.

    **Observações:** [Quaisquer observações adicionais sobre o cardápio, como dicas de preparo em lote.]
    """

    resultado = call_agent(agente, prompt)
    return resultado

In [46]:
###########################################
# --- Agente 2: Planejador de compras --- #
###########################################
def agente_planejador_compras(cardapio: str) -> str:
    agente = Agent(
        name="agente_planejador_compras",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em organização e planejamento de compras.
        Com base no cardápio semanal, crie uma lista de compras detalhada
        com as quantidades totais necessárias de cada item para preparar todas
        as refeições ao longo da semana.
        Agrupe os itens por categoria (frutas, legumes, proteínas, grãos, etc.)
        para facilitar a organização.
        """,
        description="Agente planejador de compras da semana",
        tools=[google_search]
    )

    prompt = f"""
    Crie uma lista organizada de compras por setores/categorias de alilmentos.
    Cardápio Semanal:
    [{cardapio}]

    Formato de saída desejado:

    **Lista de Compras Semanal:**

    **🍓 Frutas:**
    - [ITEM]: [QUANTIDADE]
    - [ITEM]: [QUANTIDADE]
    - ...

    **🥦 Legumes:**
    - [ITEM]: [QUANTIDADE]
    - [ITEM]: [QUANTIDADE]
    - ...

    **🥩 Proteínas:**
    - [ITEM]: [QUANTIDADE]
    - [ITEM]: [QUANTIDADE]
    - ...

    ... e assim por diante para todas as categorias de alimentos.

    **Observações:** [Quaisquer observações sobre a lista de compras, como sugestões de embalagens maiores para economia.]
    """

    # Executa o agente
    resultado = call_agent(agente, prompt)
    return resultado

In [47]:
################################
# --- Agente 3: Cozinheiro --- #
################################
def agente_cozinheiro(cardapio: str) -> str:
    agente = Agent(
        name="agente_cozinheiro",
        model="gemini-2.0-flash",
        instruction="""
            Você é um cozinheiro super experiente, especialista em receitas práticas e saudáveis.
            Você entende o quanto é difícil executar receitas complexas no
            dia-a-dia, portanto você classifica receitas para pessoas comuns
            fazerem em casa e ajuda elas a não saírem da sua dieta.
            Você utiliza como apoio receitas encontradas nos blogs e conteúdos
            da internet que você encontra utilizando o Google (google_search)
            Para cada refeição sugerida no cardápio semanal abaixo,
            apresente uma receita fácil e prática de preparar.
            Inclua os ingredientes necessários e um breve passo a passo da preparação.
            Se a receita foi encontrada na internet, inlcua o lik da receita original.
            """,
        description="Agente cozinheiro especialista em receitas práticas e saudáveis",
        tools=[google_search]
    )
    prompt = f"""
    Cardápio Semanal:
    [{cardapio}]

    Formato de saída desejado:

    **Receitas Semanais:**

    **Dia 1:**

    - **🍽️ [REFEIÇÃO 1]: [SUGESTÃO]**]
        - Receita original: [LINK]
        - **Ingredientes:**
            - [ITEM]: [QUANTIDADE]
            - [ITEM]: [QUANTIDADE]
            - ...
        - **Modo de Preparo:**
            1. [PASSO 1]
            2. [PASSO 2]
            3. ...

    - **🍽️ [REFEIÇÃO 2]: [SUGESTÃO]**
        - Receita original: [LINK]
        - **Ingredientes:**
            - [ITEM]: [QUANTIDADE]
            - [ITEM]: [QUANTIDADE]
            - ...
        - **Modo de Preparo:**
            1. [PASSO 1]
            2. [PASSO 2]
            3. ...

    ... e assim por diante para todas as refeições de todos os dias.

    **Dicas:** [Quaisquer dicas adicionais sobre o preparo das receitas, como substituições de ingredientes.]
    """
    # Executa o agente
    resultado = call_agent(agente, prompt)
    return resultado

In [50]:
 ######################################################
# --- Main: execucao dos agentes sequencialmente --- #
######################################################
print("🚀 Iniciando o Sistema de Planejamento de Cardápio de Dieta com 3 Agentes 🚀")

# --- Obter informações do Usuário
calorias_diarias = input("❓Qual a sua meta de calorias diárias? ")
preferencias = input("❓Você tem alguma preferência alimentar (vegetariano, sem glúten,sem lactose, frango, batata etc.)? (Deixe em branco se não tiver) ")
refeicoes = input("❓Quais refeições você gostaria de planejar? Separe por vírgula (padrão caso não deseje informar: café da manhã, almoço, jantar): ")
refeicoes = "café da manhã,almoço,jantar".split(",") if not refeicoes else [refeicao.strip() for refeicao in refeicoes.split(",")]

# Inserir lógica do sistema de agentes
if not calorias_diarias or not refeicoes:
    print("Você esqueceu de digitar as informações necessárias para criar o planejamento!")
    print(f"""
    Você esqueceu de digitar as informações necessárias para criar o planejamento:
    - Meta de calorias diárias: [{calorias_diarias}]
    - Preferências alimentares: [{preferencias}]
    - Refeições a serem planejadas: [{refeicoes}]
    """)
else:
    print(f"""
    Maravilha! Vamos então criar o planejamento das refeições:
    - Meta de calorias diárias: [{calorias_diarias}]
    - Preferências alimentares: [{preferencias}]
    - Refeições a serem planejadas: [{refeicoes}]
    """)
    print("--------------------------------------------------------------")

    cardapio = agente_planejador_nutricao(calorias_diarias, preferencias, refeicoes)
    print("\n📝 Resultado do Agente 1 (Nutricionista)\n")
    display(to_markdown(cardapio))
    print("--------------------------------------------------------------")

    planejamento = agente_planejador_compras(cardapio)
    print("\n📝 Resultado do Agente 2 (Planejador de compras)\n")
    display(to_markdown(planejamento))
    print("--------------------------------------------------------------")

    receitas = agente_cozinheiro(cardapio)
    print("\n📝 Resultado do Agente 3 (Cozinheiro)\n")
    display(to_markdown(receitas))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Planejamento de Cardápio de Dieta com 3 Agentes 🚀
❓Qual a sua meta de calorias diárias? 1500
❓Você tem alguma preferência alimentar (vegetariano, sem glúten,sem lactose, frango, batata etc.)? (Deixe em branco se não tiver) 
❓Quais refeições você gostaria de planejar? Separe por vírgula (padrão caso não deseje informar: café da manhã, almoço, jantar): 

   Maravilha! Vamos então criar o planejamento das refeições:
   - Meta de calorias diárias: [1500]
   - Preferências alimentares: []
   - Refeições a serem planejadas: [['café da manhã', 'almoço', 'jantar']]
   

📝 Resultado do Agente 1 (Nutricionista)



> Aqui está uma proposta de cardápio semanal de 1500 calorias, priorizando a praticidade, ingredientes acessíveis e variedade:
> 
> 
> **Cardápio Semanal:**
> 
> **📆 Dia 1:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Aveia (40g) (150 calorias);
>       - Leite desnatado (200ml) (70 calorias);
>       - Banana (1 unidade pequena) (80 calorias);
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Arroz integral cozido (100g) (110 calorias);
>       - Feijão carioca cozido (120g) (100 calorias);
>       - Frango grelhado (150g) (250 calorias);
>       - Salada de alface, tomate e pepino (à vontade) (50 calorias);
>       - Azeite extra virgem (1 colher de sopa) (90 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Sopa de legumes (cenoura, abóbora, chuchu) (300g) (150 calorias);
>       - Omelete com 2 ovos e espinafre (200 calorias);
>       - Pão integral (2 fatias) (120 calorias);
>       - Queijo branco (50g) (130 calorias)
> 
> **📆 Dia 2:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Iogurte natural desnatado (200g) (140 calorias);
>       - Frutas vermelhas (1 xícara) (80 calorias);
>       - Granola sem açúcar (30g) (80 calorias);
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Quinoa cozida (100g) (120 calorias);
>       - Lentilha cozida (150g) (190 calorias);
>       - Carne magra cozida (120g) (200 calorias);
>       - Brócolis cozido no vapor (1 xícara) (40 calorias);
>       - Azeite extra virgem (1 colher de sopa) (90 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Salada de grão de bico (150g), tomate, cebola e salsa (250 calorias);
>       - Atum em conserva light (1 lata) (130 calorias);
>       - Torradas integrais (3 unidades) (150 calorias);
>       - Abacate (50g) (70 calorias)
> 
> **📆 Dia 3:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Pão integral (2 fatias) (120 calorias);
>       - Queijo cottage (50g) (50 calorias);
>       - Ovo cozido (1 unidade) (70 calorias);
>       - Tomate (1 unidade) (10 calorias);
>       - Café sem açúcar (à vontade) (50 calorias);
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Batata doce cozida (150g) (130 calorias);
>       - Peixe assado (salmão ou tilápia) (150g) (250 calorias);
>       - Couve refogada (1 xícara) (50 calorias);
>       - Limão e ervas para temperar (à vontade) (70 calorias);
>       - Arroz integral cozido (100g) (100 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Canja de galinha com legumes (cenoura, abobrinha) (350 calorias);
>       - Pão francês (1 unidade) (140 calorias);
>       - Ricota (50g) (110 calorias);
> 
> **📆 Dia 4:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Smoothie de frutas (banana, morango, mamão) com leite vegetal (200ml) (300 calorias)
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Arroz integral (100g) (110 calorias);
>       - Feijão preto (120g) (100 calorias);
>       - Carne moída magra refogada com legumes (150g) (300 calorias);
>       - Salada de folhas verdes (à vontade) (40 calorias);
>       - Azeite extra virgem (1 colher de sopa) (50 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Sopa de ervilha com legumes (350 calorias);
>       - Torradas integrais (2 unidades) (150 calorias);
>       - Queijo magro (100 calorias);
> 
> **📆 Dia 5:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Tapioca (2 colheres de sopa) com queijo branco (50g) e chia (1 colher de chá) (300 calorias)
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Cuscuz marroquino (100g) com legumes (abobrinha, cenoura, pimentão) (350 calorias);
>       - Frango desfiado (100g) (150 calorias);
>       - Salada (100 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Pizza de frigideira integral (1 unidade) com molho de tomate caseiro, queijo e legumes (450 calorias);
>       - Salada (150 calorias);
> 
> **📆 Dia 6:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Mingau de aveia com leite desnatado, fruta picada (maçã) e canela (300 calorias)
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Escondidinho de batata doce com carne moída magra e legumes (450 calorias);
>       - Salada (150 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Wraps integrais (2 unidades) com frango desfiado, guacamole e salada (500 calorias);
>       - Frutas (100 calorias);
> 
> **📆 Dia 7:**
> - **Café da Manhã:**
>     - Total de calorias: 300 kcal;
>       - Omelete com legumes e queijo cottage (300 calorias)
> - **Almoço:**
>     - Total de calorias: 600 kcal;
>       - Macarrão integral com molho de tomate caseiro e almôndegas de carne magra (500 calorias);
>       - Salada (100 calorias);
> - **Jantar:**
>     - Total de calorias: 600 kcal;
>       - Sopa creme de abóbora com frango desfiado (400 calorias);
>       - Pão integral com requeijão light (200 calorias);
> 
> **Observações:**
> 
> *   **Preparo em Lote:** As sopas, feijão, arroz integral, quinoa, lentilha, carne moída e frango desfiado podem ser preparados em maior quantidade e armazenados na geladeira ou congelador para facilitar o consumo ao longo da semana.
> *   **Temperos:** Utilize temperos naturais como alho, cebola, ervas frescas, pimenta e azeite extra virgem para dar sabor aos pratos, evitando o uso excessivo de sal.
> *   **Lanches:** Se sentir fome entre as refeições, opte por frutas, iogurte natural, castanhas ou palitos de legumes com guacamole.
> *   **Hidratação:** Beba bastante água ao longo do dia (pelo menos 2 litros).
> *   **Variações:** Sinta-se à vontade para substituir os alimentos por outros equivalentes em termos de valor nutricional e calórico, de acordo com suas preferências e disponibilidade.
> *   **Consulta Profissional:** Este cardápio é apenas uma sugestão. É fundamental consultar um nutricionista para uma avaliação individualizada e um plano alimentar personalizado.
> *   **Ingredientes Baratos:** Priorize ingredientes da época, feiras livres e mercados municipais para encontrar melhores preços.
> 
> Espero que este cardápio seja útil para você alcançar sua meta de 1500 calorias de forma saudável e prática!
> 


--------------------------------------------------------------

📝 Resultado do Agente 2 (Planejador de compras)



> Com certeza! Aqui está uma lista de compras organizada por categorias, baseada no cardápio semanal fornecido.
> 
> 
> **Lista de Compras Semanal:**
> 
> **🍓 Frutas:**
> 
> *   Banana: 5 unidades pequenas
> *   Frutas vermelhas: 1 xícara
> *   Tomate: 1 unidade
> *   Mamão: Meio unidade
> *   Morango: A gosto
> *   Maçã: 1 unidade
> *   Abacate: 50g
> 
> **🥦 Legumes:**
> 
> *   Alface: 1 unidade
> *   Tomate: 1 unidade
> *   Pepino: 1 unidade
> *   Cenoura: 5 unidades (para sopa e canja)
> *   Abóbora: 2 pedaços pequenos (para sopa e canja)
> *   Chuchu: 1 unidade
> *   Brócolis: 1 xícara
> *   Couve: 1 xícara
> *   Cebola: 1 unidade
> *   Salsa: A gosto
> *   Abobrinha: A gosto
> *   Pimentão: A gosto
> *   Ervilhas: A gosto
> *   Legumes variados para pizza de frigideira e wraps
> 
> **🥩 Proteínas:**
> 
> *   Frango: 400g (grelhado e desfiado)
> *   Ovos: 9 unidades
> *   Atum em conserva light: 1 lata
> *   Salmão ou tilápia: 150g
> *   Carne magra moída: 300g
> *   Almôndegas de carne magra: A gosto
> 
> **🌾 Grãos e Carboidratos:**
> 
> *   Aveia em flocos: 40g
> *   Arroz integral: 300g
> *   Quinoa: 100g
> *   Pão integral: 4 fatias
> *   Torradas integrais: 5 unidades
> *   Batata doce: 150g
> *   Pão francês: 1 unidade
> *   Tapioca: 2 colheres de sopa
> *   Cuscuz marroquino: 100g
> *   Macarrão integral: A gosto
> 
> **🥛 Laticínios:**
> 
> *   Leite desnatado: 200ml
> *   Iogurte natural desnatado: 200g
> *   Queijo branco: 50g
> *   Queijo cottage: 100g
> *   Ricota: 50g
> *   Queijo magro: 100g
> *   Requeijão light: A gosto
> 
> **🌰 Sementes e Oleaginosas:**
> 
> *   Granola sem açúcar: 30g
> *   Chia: 1 colher de chá
> 
> **🥫 Outros:**
> 
> *   Feijão carioca: 120g
> *   Lentilha: 150g
> *   Grão de bico: 150g
> *   Feijão preto: 120g
> *   Azeite extra virgem
> *   Limão
> *   Ervas frescas
> *   Caldo de galinha (para a canja)
> *   Molho de tomate caseiro
> *   Guacamole
> *   Café sem açúcar
> 
> **Observações:**
> 
> *   **Preparo em Lote:** Como mencionado no cardápio, considere comprar em maior quantidade itens como feijão, arroz integral, quinoa, lentilha, carne moída e frango para otimizar o tempo de preparo ao longo da semana.
> *   **Embalagens Maiores:** Para itens como aveia, arroz integral e feijão, embalagens maiores costumam ser mais econômicas.
> *   **Substituições:** Sinta-se à vontade para substituir os ingredientes de acordo com suas preferências e o que estiver mais fresco e acessível no mercado.
> *   **Aproveitamento:** Utilize talos e folhas de legumes para preparar caldos e sopas, evitando o desperdício.


--------------------------------------------------------------

📝 Resultado do Agente 3 (Cozinheiro)



> Ok, aqui está um cardápio semanal de 1500 calorias com receitas práticas e saudáveis, além de dicas para facilitar o seu dia a dia.
> 
> 
> **Receitas Semanais:**
> 
> **Dia 1:**
> 
> - **🍽️ Café da Manhã: Aveia com Leite e Banana**
>     - Receita original: [https://www.youtube.com/watch?v=WzeZrhiBTaA](https://www.youtube.com/watch?v=WzeZrhiBTaA)
>     - **Ingredientes:**
>         - Aveia em flocos: 40g
>         - Leite desnatado: 200ml
>         - Banana pequena: 1 unidade
>     - **Modo de Preparo:**
>         1.  Em uma tigela, misture a aveia e o leite.
>         2.  Amasse a banana e adicione à mistura.
>         3.  Leve ao microondas por 2-3 minutos ou cozinhe em fogo baixo até engrossar.
> 
> - **🍽️ Almoço: Arroz Integral com Feijão e Frango Grelhado**
>     - Receita original: [https://www.receitasnestle.com.br/receitas/marmita-de-frango-grelhado-com-arroz-integral-feijao-e-legumes-ao-forno](https://www.receitasnestle.com.br/receitas/marmita-de-frango-grelhado-com-arroz-integral-feijao-e-legumes-ao-forno)
>     - **Ingredientes:**
>         - Arroz integral cozido: 100g
>         - Feijão carioca cozido: 120g
>         - Peito de frango: 150g
>         - Alface, tomate e pepino: a gosto
>         - Azeite extra virgem: 1 colher de sopa
>     - **Modo de Preparo:**
>         1.  Tempere o frango com sal, pimenta e limão.
>         2.  Grelhe o frango até dourar.
>         3.  Sirva o arroz e o feijão com o frango grelhado e a salada.
> 
> - **🍽️ Jantar: Sopa de Legumes com Omelete e Queijo**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Cenoura, abóbora, chuchu picados: 300g
>         - Ovos: 2 unidades
>         - Espinafre picado: a gosto
>         - Pão integral: 2 fatias
>         - Queijo branco: 50g
>     - **Modo de Preparo:**
>         1.  Cozinhe os legumes em água com sal até ficarem macios.
>         2.  Bata os ovos com espinafre e faça um omelete.
>         3.  Sirva a sopa com o omelete, pão integral e queijo branco.
> 
> **Dia 2:**
> 
> - **🍽️ Café da Manhã: Iogurte com Frutas Vermelhas e Granola**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Iogurte natural desnatado: 200g
>         - Frutas vermelhas (morango, amora, framboesa): 1 xícara
>         - Granola sem açúcar: 30g
>     - **Modo de Preparo:**
>         1.  Misture todos os ingredientes em uma tigela e consuma.
> 
> - **🍽️ Almoço: Quinoa com Lentilha e Carne Magra**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Quinoa cozida: 100g
>         - Lentilha cozida: 150g
>         - Carne magra cozida (patinho, músculo): 120g
>         - Brócolis cozido no vapor: 1 xícara
>         - Azeite extra virgem: 1 colher de sopa
>     - **Modo de Preparo:**
>         1.  Cozinhe a quinoa e a lentilha separadamente.
>         2.  Refogue a carne magra.
>         3.  Sirva a quinoa e a lentilha com a carne e o brócolis.
> 
> - **🍽️ Jantar: Salada de Grão de Bico com Atum e Abacate**
>     - Receita original: [https://www.nestle.com.br/receitas/salada-de-grao-de-bico-com-atum](https://www.nestle.com.br/receitas/salada-de-grao-de-bico-com-atum)
>     - **Ingredientes:**
>         - Grão de bico cozido: 150g
>         - Tomate, cebola e salsa picados: a gosto
>         - Atum em conserva light: 1 lata
>         - Torradas integrais: 3 unidades
>         - Abacate: 50g
>     - **Modo de Preparo:**
>         1.  Misture o grão de bico, tomate, cebola e salsa.
>         2.  Adicione o atum.
>         3.  Sirva a salada com torradas e abacate.
> 
> **Dia 3:**
> 
> - **🍽️ Café da Manhã: Pão Integral com Queijo Cottage e Ovo Cozido**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Pão integral: 2 fatias
>         - Queijo cottage: 50g
>         - Ovo cozido: 1 unidade
>         - Tomate: 1 unidade
>         - Café sem açúcar: à vontade
>     - **Modo de Preparo:**
>         1.  Prepare o café.
>         2.  Cozinhe o ovo.
>         3.  Monte o sanduíche com pão, queijo cottage, ovo e tomate.
> 
> - **🍽️ Almoço: Peixe Assado com Batata Doce e Couve**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Batata doce cozida: 150g
>         - Peixe assado (salmão ou tilápia): 150g
>         - Couve refogada: 1 xícara
>         - Limão e ervas: a gosto
>         - Arroz integral cozido: 100g
>     - **Modo de Preparo:**
>         1.  Asse o peixe com limão e ervas.
>         2.  Cozinhe a batata doce.
>         3.  Refogue a couve.
>         4.  Sirva o peixe com batata doce, couve e arroz integral.
> 
> - **🍽️ Jantar: Canja de Galinha com Legumes**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Canja de galinha com legumes (cenoura, abobrinha): 350 calorias
>         - Pão francês: 1 unidade
>         - Ricota: 50g
>     - **Modo de Preparo:**
>         1.  Prepare a canja com os legumes.
>         2.  Sirva a canja com pão e ricota.
> 
> **Dia 4:**
> 
> - **🍽️ Café da Manhã: Smoothie de Frutas com Leite Vegetal**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Banana, morango, mamão: a gosto
>         - Leite vegetal (amêndoa, arroz, soja): 200ml
>     - **Modo de Preparo:**
>         1.  Bata todas as frutas com o leite vegetal no liquidificador.
> 
> - **🍽️ Almoço: Arroz Integral com Feijão Preto e Carne Moída Magra com Legumes**
>     - Receita original: [https://www.tiktok.com/@panelinha.de.sabores/video/7344405368392924422](https://www.tiktok.com/@panelinha.de.sabores/video/7344405368392924422)
>     - **Ingredientes:**
>         - Arroz integral: 100g
>         - Feijão preto: 120g
>         - Carne moída magra refogada com legumes: 150g
>         - Salada de folhas verdes: à vontade
>         - Azeite extra virgem: 1 colher de sopa
>     - **Modo de Preparo:**
>         1.  Prepare o arroz e o feijão.
>         2.  Refogue a carne moída com os legumes.
>         3.  Sirva o arroz, o feijão e a carne moída com salada.
> 
> - **🍽️ Jantar: Sopa de Ervilha com Legumes**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Sopa de ervilha com legumes: 350 calorias
>         - Torradas integrais: 2 unidades
>         - Queijo magro: 100 calorias
>     - **Modo de Preparo:**
>         1.  Prepare a sopa de ervilha com legumes.
>         2.  Sirva a sopa com torradas e queijo.
> 
> **Dia 5:**
> 
> - **🍽️ Café da Manhã: Tapioca com Queijo Branco e Chia**
>     - Receita original: [https://www.receiteria.com.br/receita/tapioca-com-chia-e-queijo/](https://www.receiteria.com.br/receita/tapioca-com-chia-e-queijo/)
>     - **Ingredientes:**
>         - Tapioca: 2 colheres de sopa
>         - Queijo branco: 50g
>         - Chia: 1 colher de chá
>     - **Modo de Preparo:**
>         1.  Misture a tapioca com a chia.
>         2.  Espalhe em uma frigideira e aqueça.
>         3.  Adicione o queijo e dobre a tapioca.
> 
> - **🍽️ Almoço: Cuscuz Marroquino com Legumes e Frango Desfiado**
>     - Receita original: [https://www.youtube.com/watch?v=v67kMy-GDu8](https://www.youtube.com/watch?v=v67kMy-GDu8)
>     - **Ingredientes:**
>         - Cuscuz marroquino: 100g
>         - Legumes (abobrinha, cenoura, pimentão): a gosto
>         - Frango desfiado: 100g
>         - Salada: 100 calorias
>     - **Modo de Preparo:**
>         1.  Prepare o cuscuz conforme as instruções.
>         2.  Refogue os legumes.
>         3.  Misture o cuscuz com os legumes e o frango.
>         4.  Sirva com salada.
> 
> - **🍽️ Jantar: Pizza de Frigideira Integral com Legumes**
>     - Receita original: [https://www.youtube.com/watch?v=afwI9hnImJE](https://www.youtube.com/watch?v=afwI9hnImJE)
>     - **Ingredientes:**
>         - Farinha de trigo integral: 50g
>         - Água morna: 1/4 de xícara
>         - Sal: 1 pitada
>     - **Modo de Preparo:**
>         1. Junte todos ingredientes e misture
>         2. Em uma frigideira adicione a massa em fogo baixo até dourar dos dois lados
>         3. Adicione molho de tomate caseiro, queijo e legumes (450 calorias);
>         4.  Sirva com salada (150 calorias);
> 
> **Dia 6:**
> 
> - **🍽️ Café da Manhã: Mingau de Aveia com Maçã e Canela**
>     - Receita original: [https://www.tedi.com.br/receitas/mingau-de-aveia-com-maca-e-canela](https://www.tedi.com.br/receitas/mingau-de-aveia-com-maca-e-canela)
>     - **Ingredientes:**
>         - Aveia: 3 colheres de sopa
>         - Leite desnatado: 200ml
>         - Maçã picada: 1 unidade
>         - Canela: a gosto
>     - **Modo de Preparo:**
>         1.  Cozinhe a aveia com o leite até engrossar.
>         2.  Adicione a maçã e a canela.
> 
> - **🍽️ Almoço: Escondidinho de Batata Doce com Carne Moída Magra e Legumes**
>     - Receita original: [https://www.knorr.com.br/receitas/escondidinho-de-batata-doce-com-carne-moida.html](https://www.knorr.com.br/receitas/escondidinho-de-batata-doce-com-carne-moida.html)
>     - **Ingredientes:**
>         - Batata doce cozida e amassada: 150g
>         - Carne moída magra refogada com legumes: 450 calorias
>         - Salada: 150 calorias
>     - **Modo de Preparo:**
>         1.  Faça um purê de batata doce.
>         2.  Refogue a carne moída com legumes.
>         3.  Monte o escondidinho com purê e carne moída.
>         4.  Leve ao forno para gratinar.
> 
> - **🍽️ Jantar: Wraps Integrais com Frango Desfiado, Guacamole e Salada**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Wraps integrais: 2 unidades
>         - Frango desfiado: a gosto
>         - Guacamole: a gosto
>         - Salada: a gosto
>         - Frutas: 100 calorias
>     - **Modo de Preparo:**
>         1.  Recheie os wraps com frango, guacamole e salada.
>         2.  Sirva com frutas.
> 
> **Dia 7:**
> 
> - **🍽️ Café da Manhã: Omelete com Legumes e Queijo Cottage**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Ovos: 2 unidades
>         - Legumes picados (tomate, cebola, pimentão): a gosto
>         - Queijo cottage: a gosto
>     - **Modo de Preparo:**
>         1.  Bata os ovos com os legumes e o queijo cottage.
>         2.  Faça o omelete em uma frigideira.
> 
> - **🍽️ Almoço: Macarrão Integral com Molho de Tomate Caseiro e Almôndegas**
>     - Receita original: Não especificada, receita adaptada.
>     - **Ingredientes:**
>         - Macarrão integral cozido: a gosto
>         - Molho de tomate caseiro: a gosto
>         - Almôndegas de carne magra: 500 calorias
>         - Salada: 100 calorias
>     - **Modo de Preparo:**
>         1.  Prepare o macarrão integral.
>         2.  Prepare o molho de tomate e as almôndegas.
>         3.  Sirva o macarrão com molho e almôndegas.
>         4.  Acompanhe com salada.
> 
> - **🍽️ Jantar: Sopa Creme de Abóbora com Frango Desfiado**
>     - Receita original: [https://www.band.uol.com.br/receitas/sopa-de-abobora-com-frango-desfiado-16854537](https://www.band.uol.com.br/receitas/sopa-de-abobora-com-frango-desfiado-16854537)
>     - **Ingredientes:**
>         - Abóbora cozida e amassada: a gosto
>         - Frango desfiado: a gosto
>         - Tempero a gosto
>         - Pão integral: a gosto
>         - Requeijão light: a gosto
>     - **Modo de Preparo:**
>         1.  Refogue a abóbora e junte o frango desfiado e o creme de leite.
>         2.  Sirva com pão integral com requeijão light.
> 
> **Dicas:**
> 
> *   **Substituições:** Sinta-se à vontade para substituir os ingredientes por outros equivalentes em termos de valor nutricional e calórico, de acordo com suas preferências e disponibilidade. Por exemplo, troque o frango por peixe ou tofu, o arroz integral por quinoa, etc.
> *   **Preparo em Lote:** Aproveite para preparar grandes quantidades de arroz integral, feijão, lentilha, quinoa, frango desfiado e carne moída no fim de semana. Isso economiza tempo durante a semana.
> *   **Temperos Naturais:** Use e abuse de temperos naturais como alho, cebola, ervas frescas, pimenta e azeite extra virgem para dar sabor aos pratos, evitando o uso excessivo de sal.
> *   **Lanches:** Para os lanches entre as refeições, tenha sempre à mão opções saudáveis como frutas, iogurte natural, castanhas, palitos de legumes com guacamole ou ovo cozido.
> *   **Consulta Profissional:** É fundamental consultar um nutricionista para uma avaliação individualizada e um plano alimentar personalizado.
> 


--------------------------------------------------------------
